In [29]:
import cifar10_input
import tensorflow as tf
import numpy as np

In [30]:
batch_size = 128
data_dir = 'cifar-10-batches-bin'
images_train,labels_train = cifar10_input.inputs(eval_data=False,data_dir = data_dir,batch_size = batch_size)
images_test,labels_test = cifar10_input.inputs(eval_data=True,data_dir = data_dir,batch_size = batch_size)

In [31]:
#定义网络结构
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
def avg_pool_6x6(x):
    return tf.nn.avg_pool(x,ksize=[1,6,6,1],strides=[1,6,6,1],padding='SAME')

In [32]:
#定义占位符
x = tf.placeholder(tf.float32,[None,24,24,3])
y = tf.placeholder(tf.float32,[None,10])

W_conv1 = weight_variable([5,5,3,64])
b_conv1 = bias_variable([64])

x_image = tf.reshape(x,[-1,24,24,3])

h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5,5,64,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_conv3 = weight_variable([5,5,64,10])
b_conv3 = bias_variable([10])
h_conv3 = tf.nn.relu(conv2d(h_pool2,W_conv3) + b_conv3)

nt_hpool3 = avg_pool_6x6(h_conv3)#10
nt_hpool3_flat = tf.reshape(nt_hpool3,[-1,10])
y_conv = tf.nn.softmax(nt_hpool3_flat)

cross_entroy = -tf.reduce_sum(y*tf.log(y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entroy)
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

In [33]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
tf.train.start_queue_runners(sess=sess)
for i in range(15000):
    image_batch,label_batch = sess.run([image_train,label_train])
    label_b = np.eye(10,dtype=float)[label_batch]
    train_step.run(feed_dict={x:image_batch,y:label_b},session=sess)
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:image_batch,y:label_b},session=sess)
        print("step %d,training accuracy %g"%(i,train_accuracy))
#test
image_batch,label_batch = sess.run([image_test,label_b])
label_b = np.eye(10,dtype=float)[label_batch]
print("Finished! test accuracy %g"%accuracy.eval(feed_dict={x:image_batch,y:label_b},session=sess))

step 0,training accuracy 0.148438
step 200,training accuracy 0.34375


KeyboardInterrupt: 